In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil2 as teo
import useful_rid_code as rid

sn.set_context("notebook")

# RID

## Estimating loads in unmonitored regions (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

The [TEOTIL2 model](https://nivanorge.github.io/teotil2/) is used to estimate loads in unmonitored areas. We know the regine ID for each of the 155 stations where water chemistry is measured, and we also know which OSPAR region each monitoring site drains to. We want to use observed data to estimate loads upstream of each monitoring point, and modelled data elsewhere.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 2009


## 1. Generate model input file

In [4]:
# Parameters of interest
par_list = ["Tot-N", "Tot-P"]

# Path to TEOTIL2 "core" input data
teo_fold = r"../../../teotil2/data/core_input_data"

# Ouput path for model file
ann_input_csv = f"../../../teotil2/data/norway_annual_input_data/input_data_{year}.csv"

In [5]:
engine = rid.connect_to_nivabase(user=user, pw=pw)
df = teo.io.make_input_file(
    year, engine, teo_fold, ann_input_csv, mode="nutrients", par_list=par_list
)

Connection successful.


## 2. Run model

In [6]:
%%time

# Run model
g = teo.model.run_model(ann_input_csv)

/opt/conda/lib/python3.9/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


CPU times: user 8.72 s, sys: 72.5 ms, total: 8.8 s
Wall time: 8.83 s


## 3. Save results

In [7]:
# Save results as csv
res_csv = f"../../../teotil2/data/norway_annual_output_data/teotil2_results_{year}.csv"
df = teo.model.model_to_dataframe(g, out_path=res_csv)

df.head()

,regine,regine_ned,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,...,local_ren_tot-n_tonnes,local_ren_tot-p_tonnes,local_runoff_mm/yr,local_spr_tot-n_tonnes,local_spr_tot-p_tonnes,local_trans_tot-n,local_trans_tot-p,local_urban_tot-n_tonnes,local_urban_tot-p_tonnes,local_vol_lake_m3
0,001.1A2B,001.1A2A,2.593994,0.147263,0.033365,0.002762,0.300907,0.037346,13.819028,0.510816,...,0.0,0.0,337.680823,0.267543,0.034584,1.00,1.00,0.02205,0.00315,1.788754e+08
1,001.1A4D,001.1A4C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.496581,0.009718,...,0.0,0.0,393.960960,0.000000,0.000000,0.81,0.26,0.00000,0.00000,8.019497e+06
2,001.1M,001.1L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.584771,0.034180,...,0.0,0.0,393.960960,0.000000,0.000000,0.86,0.31,0.00000,0.00000,3.866908e+07
3,001.221Z,001.2210,0.104655,0.005941,0.001346,0.000111,0.012140,0.001507,0.415372,0.010737,...,0.0,0.0,337.680823,0.010794,0.001395,1.00,1.00,0.00000,0.00000,0.000000e+00
4,001.222Z,001.2220,0.862410,0.048960,0.011093,0.000918,0.100041,0.012416,2.133435,0.075585,...,0.0,0.0,337.680823,0.088948,0.011498,1.00,1.00,0.00000,0.00000,0.000000e+00


In [8]:
# Save version with main catchments only
main_list = ["%03d." % i for i in range(1, 316)]
df2 = df.query("regine in @main_list")
df2.sort_values("regine", inplace=True)

# Save
main_csv = f"../../../Results/Unmon_loads/teotil2_results_{year}_main_catchs.csv"
df2.to_csv(main_csv, index=False, encoding="utf-8")

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## 4. Explore results

### 4.1. Total N and P

####  4.1.1. Identify areas with monitoring data

Where observations are available, we want to use them in preference to the model output. This means identifying all the catchments with observed data and substracting the model results for these locations. This is more complicated than it appears, because a small number of observed catchments are upstream of others, so subtracting all the loads for the 155 monitored catchments involves "double accounting", which we want to avoid. The first step is therefore to identify the downstream-most nodes for the monitored areas i.e. for the cases where one catchment is upstream of another, we just want the downstream node.

In [9]:
# Read station data
in_xlsx = r"../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")
stn_df = stn_df.query("station_id != 38005")  # Ignore TROEMÅL2

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df[["ospar_region", "nve_vassdrag_nr"]].drop_duplicates()

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network from input file
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

#### 4.1.2. Sum model results for monitored locations

In [10]:
# Read model output
teo_df = pd.read_csv(res_csv)

# Join accumulated outputs to stns of interest
mon_df = pd.merge(
    stn_df, teo_df, how="left", left_on="nve_vassdrag_nr", right_on="regine"
)

# Groupby OSPAR region
mon_df = mon_df.groupby("ospar_region").sum()

# Get just accum cols
cols = [i for i in mon_df.columns if i.split("_")[0] == "accum"]
mon_df = mon_df[cols]

mon_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,146.253126,5.012680,2.540371,0.213926,80.885749,5.486224,3217.203502,49.656522,146.253126,5.012680,...,2990.064628,39.157618,675.559788,28.991388,0.962948,49.353990,4.309350,63555.61,0.000000,0.000000
NORTH SEA,2909.733084,64.638681,34.943085,2.173150,341.538016,47.252372,11445.674060,180.248775,2915.705655,65.182157,...,8188.430389,67.814246,1148.035615,170.559165,31.176902,127.278634,9.695664,23353.19,5.972570,0.543476
NORWEGIAN SEA2,2930.025355,74.297945,40.709881,3.053550,495.248011,40.485846,11873.860571,234.350599,2946.492333,76.427096,...,8432.120227,117.437658,1578.466172,262.783647,14.804310,176.727763,16.201886,45896.63,16.466978,2.129151
SKAGERAK,10925.936724,191.556292,102.034032,5.504629,3489.606659,97.632969,28999.614512,436.259412,11087.871963,209.034055,...,14422.135890,129.592388,1832.923662,2405.641027,15.529373,817.671697,50.803534,93945.27,161.935239,17.477763


This table gives the **modelled** inputs to each OSPAR region from catchments for which we have observed data. We want to subtract these values from the overall modelled inputs to each region and substitute the observed data instead.

The trickiest part of this is that the OSPAR regions in the TEOTIL catchment network (files named `regine_{year}.csv`) don't exactly match the relevant OSPAR definitions for this analysis. This is because the "OSPAR boundaries" in the model include catchments draining to Sweden (as part of TEOTIL2 Metals - see [here](https://nivanorge.github.io/teotil2/pages/07_1000_lakes.html)), so instead of using them directly we need to aggregate based on vassdragsnummers.

#### 4.1.3. Group model output according to "new" OSPAR regions

In [11]:
# Define "new" OSPAR regions (ranges are inclusive)
os_dict = {
    "SKAGERAK": (1, 23),
    "NORTH SEA": (24, 90),
    "NORWEGIAN SEA2": (91, 170),
    "LOFOTEN-BARENTS SEA": (171, 247),
}

# Container for results
df_list = []

# Loop over model output
for reg in os_dict.keys():
    min_id, max_id = os_dict[reg]

    regs = ["%03d." % i for i in range(min_id, max_id + 1)]

    # Get data for this region
    df2 = teo_df[teo_df["regine"].isin(regs)]

    # Get just accum cols
    cols = [i for i in df2.columns if i.split("_")[0] == "accum"]
    df2 = df2[cols]

    # Add region
    df2["ospar_region"] = reg

    # Add to output
    df_list.append(df2)

# Build df
os_df = pd.concat(df_list, axis=0)

# Aggregate
os_df = os_df.groupby("ospar_region").sum()

os_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,614.439148,25.602367,10.569251,0.866142,10897.243312,1949.788210,18800.282803,2090.621383,624.411457,27.006207,...,7278.628033,113.826965,1830.762824,928.109401,147.327780,312.589091,38.137203,138090.89,9.972310,1.403841
NORTH SEA,6965.332156,176.909329,80.515432,5.832951,20027.708752,3407.547127,43921.013049,3747.073982,7005.149638,181.977999,...,16888.154658,157.548856,2450.397778,3050.954920,327.368861,740.852512,83.669384,59314.38,39.817482,5.068670
NORWEGIAN SEA2,7927.365724,238.332570,101.652087,8.012104,22742.961575,3910.549725,46478.206118,4385.348815,7959.524515,242.573012,...,15775.720028,232.226077,3150.938771,2391.723543,288.763585,682.875005,80.786930,113934.05,32.158791,4.240442
SKAGERAK,12533.981513,246.930735,112.948837,6.321836,9183.621077,291.121661,37619.634915,713.083433,12807.715982,279.662701,...,15628.297856,142.299071,1948.666612,6897.534383,97.201257,1001.940785,71.103688,102574.69,273.734469,32.731966


We can now calculate the unmonitored component by simply subtracting the values modelled upstream of monitoring stations from the overall modelled inputs to each OSPAR region.

#### 4.1.4. Estimate loads in unmonitored areas

In [12]:
# Calc unmonitored loads
unmon_df = os_df - mon_df

# Write output
out_csv = f"../../../Results/Unmon_loads/teotil2_raw_unmonitored_loads_{year}.csv"
unmon_df.to_csv(out_csv, encoding="utf-8", index_label="ospar_region")

unmon_df.round(0).astype(int).T

ospar_region,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
accum_agri_diff_tot-n_tonnes,468,4056,4997,1608
accum_agri_diff_tot-p_tonnes,21,112,164,55
accum_agri_pt_tot-n_tonnes,8,46,61,11
accum_agri_pt_tot-p_tonnes,1,4,5,1
accum_all_point_tot-n_tonnes,10816,19686,22248,5694
accum_all_point_tot-p_tonnes,1944,3360,3870,193
accum_all_sources_tot-n_tonnes,15583,32475,34604,8620
accum_all_sources_tot-p_tonnes,2041,3567,4151,277
accum_anth_diff_tot-n_tonnes,478,4089,5013,1720
accum_anth_diff_tot-p_tonnes,22,117,166,71


#### 4.1.5. Aggregate values to required quantities

In [13]:
# Aggregate to match report
unmon_df["flow"] = unmon_df["accum_q_m3/s"] * 60 * 60 * 24 / 1000.0  # 1000s m3/day

unmon_df["sew_n"] = (
    unmon_df["accum_ren_tot-n_tonnes"] + unmon_df["accum_spr_tot-n_tonnes"]
)
unmon_df["sew_p"] = (
    unmon_df["accum_ren_tot-p_tonnes"] + unmon_df["accum_spr_tot-p_tonnes"]
)

unmon_df["ind_n"] = unmon_df["accum_ind_tot-n_tonnes"]
unmon_df["ind_p"] = unmon_df["accum_ind_tot-p_tonnes"]

unmon_df["fish_n"] = unmon_df["accum_aqu_tot-n_tonnes"]
unmon_df["fish_p"] = unmon_df["accum_aqu_tot-p_tonnes"]

unmon_df["diff_n"] = (
    unmon_df["accum_anth_diff_tot-n_tonnes"] + unmon_df["accum_nat_diff_tot-n_tonnes"]
)
unmon_df["diff_p"] = (
    unmon_df["accum_anth_diff_tot-p_tonnes"] + unmon_df["accum_nat_diff_tot-p_tonnes"]
)

new_df = unmon_df[
    ["flow", "sew_n", "sew_p", "ind_n", "ind_p", "fish_n", "fish_p", "diff_n", "diff_p"]
]

# Total for Norway
new_df.loc["NORWAY"] = new_df.sum(axis=0)

# Reorder rows
new_df = new_df.reindex(
    ["NORWAY", "LOFOTEN-BARENTS SEA", "NORTH SEA", "NORWEGIAN SEA2", "SKAGERAK"]
)

new_df.round().astype(int)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p
ospar_region,,,,,,,,,
NORWAY,358195,11968,991,2240,279,44112,8088,32839,667
LOFOTEN-BARENTS SEA,99810,1162,180,0,0,9646,1763,4767,97
NORTH SEA,112524,3494,370,437,110,15710,2877,12789,207
NORWEGIAN SEA2,135862,2635,339,825,84,18727,3443,12357,281
SKAGERAK,10000,4676,102,978,85,29,5,2926,83


## 5. Other N and P species

Tore's procedure `RESA2.FIXTEOTILPN` defines simple correction factors for estimating PO4, NO3 and NH4 from total P and N. The table below lists the factors used.

|   Source    | Phosphate | Nitrate | Ammonium |
|:-----------:|:---------:|:-------:|:--------:|
|    Sewage   |     0.600 |   0.050 |    0.750 |
|   Industry  |     0.600 |   0.050 |    0.750 |
| Aquaculture |     0.690 |   0.110 |    0.800 |
|   Diffuse   |     0.246 |   0.625 |    0.055 |


In [14]:
# Dict of conversion factors
con_dict = {
    ("sew", "po4"): ("p", 0.6),
    ("ind", "po4"): ("p", 0.6),
    ("fish", "po4"): ("p", 0.69),
    ("diff", "po4"): ("p", 0.246),
    ("sew", "no3"): ("n", 0.05),
    ("ind", "no3"): ("n", 0.05),
    ("fish", "no3"): ("n", 0.11),
    ("diff", "no3"): ("n", 0.625),
    ("sew", "nh4"): ("n", 0.75),
    ("ind", "nh4"): ("n", 0.75),
    ("fish", "nh4"): ("n", 0.8),
    ("diff", "nh4"): ("n", 0.055),
}

# Apply factors
for src in ["sew", "ind", "fish", "diff"]:
    for spc in ["po4", "no3", "nh4"]:
        el, fac = con_dict[(src, spc)]
        new_df[src + "_" + spc] = fac * new_df[src + "_" + el]

new_df.round().astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,358195,99810,112524,135862,10000
sew_n,11968,1162,3494,2635,4676
sew_p,991,180,370,339,102
ind_n,2240,0,437,825,978
ind_p,279,0,110,84,85
fish_n,44112,9646,15710,18727,29
fish_p,8088,1763,2877,3443,5
diff_n,32839,4767,12789,12357,2926
diff_p,667,97,207,281,83
sew_po4,595,108,222,203,61


## 6. Other quantities

The model currently only considers N and P, but the project focuses on a wider range of parameters. For now, we simply assume that all measured inputs (`renseanlegg`, `industri` and `akvakultur`) for regines outside of catchments with measured data make it to the sea.

We only want data for catchments that are not monitored i.e. for regine IDs **not** in the graph created above.

In [15]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
    "  a.type, "
    "  b.name, "
    "  b.unit, "
    "  SUM(c.value * d.factor) as value "
    "FROM RESA2.RID_PUNKTKILDER a, "
    "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
    "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
    "RESA2.RID_PUNKTKILDER_INP_OUTP d "
    "WHERE a.anlegg_nr = c.anlegg_nr "
    "AND (1, a.regine) NOT IN %s "
    "AND d.in_pid = c.inp_par_id "
    "AND d.out_pid = b.out_pid "
    "AND c.year = %s "
    "GROUP BY SUBSTR(a.regine, 1, 3), a.type, b.name, b.unit "
    "ORDER BY SUBSTR(a.regine, 1, 3), a.type" % (tuple(nd_list_hack), year)
)

df = pd.read_sql(sql, engine)

# Tidy
df["par"] = df["type"] + "_" + df["name"] + "_" + df["unit"]
del df["name"], df["unit"], df["type"]

# Pivot
df = df.pivot(index="vassdrag", columns="par", values="value")
df.reset_index(inplace=True)

Connection successful.


In [16]:
def f(x):
    try:
        a = int(x)
        return a
    except:
        return -999


# Convert vassdrag to numbers
df["vass"] = df["vassdrag"].apply(f)

# Get just the main catchments
df = df.query("vass != -999")

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PAH_tonn,...,RENSEANLEGG_Hg_tonn,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass
0,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,106.53385,1.75399,NaN,1
1,002,0.009123,0.007374,NaN,6.566304,0.005957,NaN,NaN,0.885828,0.300000,...,0.000150,0.28796,0.00522,0.000110,0.01414,0.056,492.35094,9.70804,0.91107,2
2,003,NaN,0.000016,NaN,NaN,0.000002,NaN,NaN,NaN,0.000068,...,0.000023,0.03594,0.00092,0.000046,0.00705,NaN,235.32600,3.06700,0.16800,3
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000007,0.00682,NaN,NaN,0.00081,NaN,115.71620,1.04903,0.04700,4
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,34.23408,0.33506,NaN,5


In [17]:
def f2(x):
    if x in range(1, 24):
        return "SKAGERAK"
    elif x in range(24, 91):
        return "NORTH SEA"
    elif x in range(91, 171):
        return "NORWEGIAN SEA2"
    elif x in range(171, 248):
        return "LOFOTEN-BARENTS SEA"
    else:
        return np.nan


# Assign main catchments to OSPAR regions
df["osp_reg"] = df["vass"].apply(f2)

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PAH_tonn,...,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass,osp_reg
0,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,106.53385,1.75399,NaN,1,SKAGERAK
1,002,0.009123,0.007374,NaN,6.566304,0.005957,NaN,NaN,0.885828,0.300000,...,0.28796,0.00522,0.000110,0.01414,0.056,492.35094,9.70804,0.91107,2,SKAGERAK
2,003,NaN,0.000016,NaN,NaN,0.000002,NaN,NaN,NaN,0.000068,...,0.03594,0.00092,0.000046,0.00705,NaN,235.32600,3.06700,0.16800,3,SKAGERAK
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00682,NaN,NaN,0.00081,NaN,115.71620,1.04903,0.04700,4,SKAGERAK
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,34.23408,0.33506,NaN,5,SKAGERAK


In [18]:
# Group by OSPAR region
df.fillna(0, inplace=True)
df = df.groupby("osp_reg").sum()
if 0 in df.index:
    df.drop(0, inplace=True)

# Total for Norway
df.loc["NORWAY"] = df.sum(axis=0)

# Join to model results
df = new_df.join(df)

# Get cols of interest
umod_cols = ["S.P.M.", "TOC", "As", "Pb", "Cd", "Cu", "Zn", "Ni", "Cr", "Hg"]
umod_cols = [
    "%s_%s_tonn" % (i, j) for i in ["INDUSTRI", "RENSEANLEGG"] for j in umod_cols
]
cols = list(new_df.columns) + umod_cols
cols.remove("RENSEANLEGG_TOC_tonn")
cols = [i for i in cols if i in df.columns]
df = df[cols]

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,358195,99810,112524,135862,10000
sew_n,11968,1162,3494,2635,4676
sew_p,991,180,370,339,102
ind_n,2240,0,437,825,978
ind_p,279,0,110,84,85
fish_n,44112,9646,15710,18727,29
fish_p,8088,1763,2877,3443,5
diff_n,32839,4767,12789,12357,2926
diff_p,667,97,207,281,83
sew_po4,595,108,222,203,61


## 7. Fish farm copper

Finally, we need to add in the Cu totals from fish farms. The method is similar to that used above, but simpler because we're only interested in one parameter.

In [19]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(regine, 1, 3) AS vassdrag, "
    "  SUM(value) AS value FROM ( "
    "    SELECT b.regine, "
    "           c.name, "
    "           (a.value*d.factor) AS value "
    "    FROM resa2.rid_kilder_aqkult_values a, "
    "    resa2.rid_kilder_aquakultur b, "
    "    resa2.rid_punktkilder_outpar_def c, "
    "    resa2.rid_punktkilder_inp_outp d "
    "    WHERE a.anlegg_nr = b.nr "
    "    AND (1, b.regine) NOT IN %s "
    "    AND a.inp_par_id = d.in_pid "
    "    AND c.out_pid = d.out_pid "
    "    AND name = 'Cu' "
    "    AND ar = %s) "
    "GROUP BY SUBSTR(regine, 1, 3)" % (tuple(nd_list_hack), year)
)

aq_df = pd.read_sql(sql, engine)

if len(aq_df) > 0:
    # Get vassdrag
    aq_df["vass"] = aq_df["vassdrag"].apply(f)
    aq_df = aq_df.query("vass != -999")

    # Calc OSPAR region and group
    aq_df["osp_reg"] = aq_df["vass"].apply(f2)
    aq_df.fillna(0, inplace=True)
    aq_df = aq_df.groupby("osp_reg").sum()
    del aq_df["vass"]

    # Total for Norway
    aq_df.loc["NORWAY"] = aq_df.sum(axis=0)

    # Rename
    aq_df.columns = [
        "AQUAKULTUR_Cu_tonn",
    ]

    # Join model results
    df = df.join(aq_df)

    df.round(0).astype(int).T

Connection successful.


In [20]:
# Write output
out_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
df.to_csv(out_csv)

This data can then be used to create Table 3 in the report - see [this notebook](https://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2017.ipynb) for details.